In [1]:
import pandas as pd
import datetime as dt

### Préparation du fichier pour la table Player dans notre BDD

In [2]:
# Chargement du fichier avec les stats par joueur
df_stats = pd.read_excel(
    "../data/raw/regular_NBA.xlsx",
    sheet_name="Données NBA", header=1)
# Chargement du fichier avec le nom des équipes
df_name_team = pd.read_excel(
    "../data/raw/regular_NBA.xlsx",
    sheet_name="Equipe")
#Chargement du fichier des définitions des colonnes de stats (pour intégration plus tard dans les métadonnées)
df_dict_stats = pd.read_excel(
    "../data/raw/regular_NBA.xlsx",
    sheet_name="Dictionnaire des données")

#### Regardons ce que nous avons comme colonne

In [3]:
df = df_stats.copy()
df.columns

Index([     'Player',        'Team',         'Age',          'GP',
                 'W',           'L',         'Min',         'PTS',
               'FGM',         'FGA',         'FG%',      15:00:00,
               '3PA',         '3P%',         'FTM',         'FTA',
               'FT%',        'OREB',        'DREB',         'REB',
               'AST',         'TOV',         'STL',         'BLK',
                'PF',          'FP',         'DD2',         'TD3',
               '+/-',      'OFFRTG',      'DEFRTG',      'NETRTG',
              'AST%',      'AST/TO',   'AST RATIO',       'OREB%',
             'DREB%',        'REB%',    'TO RATIO',        'EFG%',
               'TS%',        'USG%',        'PACE',         'PIE',
              'POSS', 'Unnamed: 45', 'Unnamed: 46', 'Unnamed: 47',
       'Unnamed: 48', 'Unnamed: 49', 'Unnamed: 50', 'Unnamed: 51',
       'Unnamed: 52'],
      dtype='object')

In [4]:
df = df[['Player','Age','Team']]
df

,Player,Age,Team
0,Shai Gilgeous-Alexander,26,OKC
1,Anthony Edwards,23,MIN
2,Nikola Jokić,30,DEN
3,Giannis Antetokounmpo,30,MIL
4,Jayson Tatum,27,BOS
...,...,...,...
564,Mac McClung,26,ORL
565,Terence Davis,28,SAC
566,Terry Taylor,25,SAC
567,Riley Minix,24,SAS


- On a bien que les colonnes qui nous intéressent

#### Regardons ce que nous avons pour les équipes

In [5]:
df_name_team

,Code,Nom complet de l'équipe
0,ATL,Atlanta Hawks
1,BKN,Brooklyn Nets
2,BOS,Boston Celtics
3,CHA,Charlotte Hornets
4,CHI,Chicago Bulls
5,CLE,Cleveland Cavaliers
6,DAL,Dallas Mavericks
7,DEN,Denver Nuggets
8,DET,Detroit Pistons
9,GSW,Golden State Warriors


#### On transforme le noms des équipes en dictionnaire afin de les mapper dans notre df final

In [6]:
team_dict = dict(zip(df_name_team["Code"], df_name_team["Nom complet de l'équipe"]))

In [7]:
df["Team_full_name"] = df["Team"].map(team_dict)
df_player = df

In [8]:
df_player

,Player,Age,Team,Team_full_name
0,Shai Gilgeous-Alexander,26,OKC,Oklahoma City Thunder
1,Anthony Edwards,23,MIN,Minnesota Timberwolves
2,Nikola Jokić,30,DEN,Denver Nuggets
3,Giannis Antetokounmpo,30,MIL,Milwaukee Bucks
4,Jayson Tatum,27,BOS,Boston Celtics
...,...,...,...,...
564,Mac McClung,26,ORL,Orlando Magic
565,Terence Davis,28,SAC,Sacramento Kings
566,Terry Taylor,25,SAC,Sacramento Kings
567,Riley Minix,24,SAS,San Antonio Spurs


- On voit rapidement que les noms des équipes sont correctement paramétrées

### Préparation du fichier pour la table Stats dans notre BDD

In [9]:
df_stats

,Player,Team,Age,GP,W,L,Min,PTS,FGM,FGA,...,PIE,POSS,Unnamed: 45,Unnamed: 46,Unnamed: 47,Unnamed: 48,Unnamed: 49,Unnamed: 50,Unnamed: 51,Unnamed: 52
0,Shai Gilgeous-Alexander,OKC,26,76,63,13,34.2,2485,859,1657,...,19.9,5514,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Anthony Edwards,MIN,23,79,48,31,36.3,2180,719,1612,...,14.0,5883,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Nikola Jokić,DEN,30,70,46,24,36.7,2072,784,1365,...,20.6,5471,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Giannis Antetokounmpo,MIL,30,67,40,27,34.2,2037,791,1320,...,21.0,4772,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Jayson Tatum,BOS,27,72,53,19,36.4,1930,662,1462,...,15.8,5368,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,Mac McClung,ORL,26,2,1,1,5.1,0,0,2,...,0.0,21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
565,Terence Davis,SAC,28,1,1,0,7.9,0,0,2,...,0.0,15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
566,Terry Taylor,SAC,25,3,1,2,2.1,0,0,1,...,4.3,14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
567,Riley Minix,SAS,24,1,0,1,6.8,0,0,1,...,3.9,15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Check des valeurs manquantes

In [10]:
df_stats.isnull().mean()

Player         0.0
Team           0.0
Age            0.0
GP             0.0
W              0.0
L              0.0
Min            0.0
PTS            0.0
FGM            0.0
FGA            0.0
FG%            0.0
15:00:00       0.0
3PA            0.0
3P%            0.0
FTM            0.0
FTA            0.0
FT%            0.0
OREB           0.0
DREB           0.0
REB            0.0
AST            0.0
TOV            0.0
STL            0.0
BLK            0.0
PF             0.0
FP             0.0
DD2            0.0
TD3            0.0
+/-            0.0
OFFRTG         0.0
DEFRTG         0.0
NETRTG         0.0
AST%           0.0
AST/TO         0.0
AST RATIO      0.0
OREB%          0.0
DREB%          0.0
REB%           0.0
TO RATIO       0.0
EFG%           0.0
TS%            0.0
USG%           0.0
PACE           0.0
PIE            0.0
POSS           0.0
Unnamed: 45    1.0
Unnamed: 46    1.0
Unnamed: 47    1.0
Unnamed: 48    1.0
Unnamed: 49    1.0
Unnamed: 50    1.0
Unnamed: 51    1.0
Unnamed: 52 

- On voit qu'il y a quelques colonnes vides

#### Suppression des colonnes vides

#### On va en profiter pour supprimer les colonnes que nous retrouvons désormais dans le dataframe Team
- Age
- Team
- Le lien entre les deux df se fera via le nom du joueur

In [11]:
delet_cols = ['Team','Age','Unnamed: 45','Unnamed: 46', 'Unnamed: 47','Unnamed: 48', 'Unnamed: 49', 'Unnamed: 50', 'Unnamed: 51','Unnamed: 52']

In [12]:
df_stats_clean = df_stats.drop(columns=delet_cols)
df_stats_clean

,Player,GP,W,L,Min,PTS,FGM,FGA,FG%,15:00:00,...,OREB%,DREB%,REB%,TO RATIO,EFG%,TS%,USG%,PACE,PIE,POSS
0,Shai Gilgeous-Alexander,76,63,13,34.2,2485,859,1657,51.9,160,...,2.5,11.2,6.9,7.0,56.9,63.7,33.6,101.91,19.9,5514
1,Anthony Edwards,79,48,31,36.3,2180,719,1612,44.7,324,...,2.1,13.1,7.7,10.3,54.7,59.5,30.7,98.27,14.0,5883
2,Nikola Jokić,70,46,24,36.7,2072,784,1365,57.6,140,...,8.3,25.2,17.3,9.2,62.7,66.3,28.5,101.99,20.6,5471
3,Giannis Antetokounmpo,67,40,27,34.2,2037,791,1320,60.1,13,...,6.7,26.6,17.2,9.1,60.7,62.5,34.6,100.18,21.0,4772
4,Jayson Tatum,72,53,19,36.4,1930,662,1462,45.2,252,...,1.7,20.6,11.2,9.2,53.7,58.2,30.1,98.14,15.8,5368
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,Mac McClung,2,1,1,5.1,0,0,2,0.0,0,...,0.0,10.0,5.0,0.0,0.0,0.0,8.7,98.50,0.0,21
565,Terence Davis,1,1,0,7.9,0,0,2,0.0,0,...,0.0,11.1,5.9,0.0,0.0,0.0,11.1,91.33,0.0,15
566,Terry Taylor,3,1,2,2.1,0,0,1,0.0,0,...,25.0,0.0,8.3,0.0,0.0,0.0,6.7,106.11,4.3,14
567,Riley Minix,1,0,1,6.8,0,0,1,0.0,0,...,0.0,20.0,11.8,0.0,0.0,0.0,6.3,106.67,3.9,15


#### Le nom de certaines colonnes risquent d'être problématique lors de la création des tables, nous allons donc en modifier certaines

In [13]:
df_stats_new = df_stats_clean.rename(columns={
    "FG%":"FG_P",
    # Format spécial pour cette colonne qui est en datatime
    dt.time(15, 0): "Min_15",
    "3PA":"PTS_3",
    "3P%":"PTS_3_P",
    "FT%":"FT_P",
    "+/-":"PLUS_MINUS",
    "AST%":"AST_P",
    "AST/TO":"AST_TO",
    "AST RATIO":"AST_RATIO",
    "OREB%":"OREB_P",
    "DREB%":"DREB_P",
    "REB%":"REB_P",
    "TO RATIO":"TO_RATIO",
    "EFG%":"EFG_P",
    "TS%":"TS_P",
    "USG%":"USG_P"
})

In [14]:
df_stats_new

,Player,GP,W,L,Min,PTS,FGM,FGA,FG_P,Min_15,...,OREB_P,DREB_P,REB_P,TO_RATIO,EFG_P,TS_P,USG_P,PACE,PIE,POSS
0,Shai Gilgeous-Alexander,76,63,13,34.2,2485,859,1657,51.9,160,...,2.5,11.2,6.9,7.0,56.9,63.7,33.6,101.91,19.9,5514
1,Anthony Edwards,79,48,31,36.3,2180,719,1612,44.7,324,...,2.1,13.1,7.7,10.3,54.7,59.5,30.7,98.27,14.0,5883
2,Nikola Jokić,70,46,24,36.7,2072,784,1365,57.6,140,...,8.3,25.2,17.3,9.2,62.7,66.3,28.5,101.99,20.6,5471
3,Giannis Antetokounmpo,67,40,27,34.2,2037,791,1320,60.1,13,...,6.7,26.6,17.2,9.1,60.7,62.5,34.6,100.18,21.0,4772
4,Jayson Tatum,72,53,19,36.4,1930,662,1462,45.2,252,...,1.7,20.6,11.2,9.2,53.7,58.2,30.1,98.14,15.8,5368
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,Mac McClung,2,1,1,5.1,0,0,2,0.0,0,...,0.0,10.0,5.0,0.0,0.0,0.0,8.7,98.50,0.0,21
565,Terence Davis,1,1,0,7.9,0,0,2,0.0,0,...,0.0,11.1,5.9,0.0,0.0,0.0,11.1,91.33,0.0,15
566,Terry Taylor,3,1,2,2.1,0,0,1,0.0,0,...,25.0,0.0,8.3,0.0,0.0,0.0,6.7,106.11,4.3,14
567,Riley Minix,1,0,1,6.8,0,0,1,0.0,0,...,0.0,20.0,11.8,0.0,0.0,0.0,6.3,106.67,3.9,15


## Préparation d'un df propre pour intégration dans les métadonnées de la définition des colonnes dans stats.

In [15]:
df_dict_stats.head()

,Dictionnaire des données,Unnamed: 1
0,Player,Nom du joueur
1,Team,Équipe du joueur (code à 3 lettres)
2,Age,Âge du joueur
3,GP,Nombre de matchs joués (Games Played)
4,W,Nombre de victoires de l'équipe lors des match...


#### On va déjà renommer le nom de la colonne qui est absente pour le moment

In [16]:
df_dict_stats_transform = df_dict_stats.copy()
df_dict_stats_transform = df_dict_stats_transform.rename(columns={
    'Unnamed: 1':'Definition_acronyme'
})
df_dict_stats_transform.head()

,Dictionnaire des données,Definition_acronyme
0,Player,Nom du joueur
1,Team,Équipe du joueur (code à 3 lettres)
2,Age,Âge du joueur
3,GP,Nombre de matchs joués (Games Played)
4,W,Nombre de victoires de l'équipe lors des match...


In [20]:
df_dict_clean = df_dict_stats_transform.set_index("Dictionnaire des données").rename({
    "FG%":"FG_P",
    # Format spécial pour cette colonne qui est en datatime
    dt.time(15, 0): "Min_15",
    "3PA":"PTS_3",
    "3P%":"PTS_3_P",
    "FT%":"FT_P",
    "+/-":"PLUS_MINUS",
    "AST%":"AST_P",
    "AST/TO":"AST_TO",
    "AST RATIO":"AST_RATIO",
    "OREB%":"OREB_P",
    "DREB%":"DREB_P",
    "REB%":"REB_P",
    "TO RATIO":"TO_RATIO",
    "EFG%":"EFG_P",
    "TS%":"TS_P",
    "USG%":"USG_P"
},axis=0)
df_dict_clean = df_dict_clean.reset_index()

In [21]:
df_dict_clean

,Dictionnaire des données,Definition_acronyme
0,Player,Nom du joueur
1,Team,Équipe du joueur (code à 3 lettres)
2,Age,Âge du joueur
3,GP,Nombre de matchs joués (Games Played)
4,W,Nombre de victoires de l'équipe lors des match...
5,L,Nombre de défaites
6,Min,Minutes moyennes jouées par match
7,PTS,Points marqués en moyenne par match
8,FGM,Tirs réussis par match (Field Goals Made)
9,FGA,Tirs tentés par match (Field Goals Attempted)
